# Section 1 - Setup
필요한 라이브러리를 가져오고, 변수를 설정하고, 노트북과 함께 제공되는 XGBoost 고객 이탈 모델을 학습하는 데 사용된 데이터를 검사합니다.
* 모델을 호스팅하는 데 사용되는 AWS region 설정
* Sagemaker 노트북 인스턴스와 연결된 IAM Role 설정
* 모델 훈련에 사용되는 데이터, 테스트 데이터 및 모델 호출에서 캡쳐한 데이터를 저장하는 데 사용되는 S3 버킷 설정

### 1.1 Import libraries

In [4]:
from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

In [5]:
import os
os.chdir('/home/ec2-user/SageMaker/model_quality_2021-11-03')

### 1.2 AWS region and  IAM Role

In [6]:
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

RoleArn: arn:aws:iam::023376671800:role/service-role/AmazonSageMaker-ExecutionRole-20211029T114568
Region: ap-northeast-2


### 1.3 S3 bucket and prefixes

In [7]:
##Setup S3 bucket
bucket = session.default_bucket()
print("Demo Bucket:", bucket)
prefix = "sagemaker/ModelMonitor"

Demo Bucket: sagemaker-ap-northeast-2-023376671800


In [8]:
##S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Capture path: s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/datacapture
Ground truth path: s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/ground_truth_data/2021-11-08-08
Report path: s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/reports


In [9]:
##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)

Image URI: 709848358524.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-model-monitor-analyzer


### 1.4 Test access to the S3 bucket
노트북에 위에 지정된 S3 버킷에 액세스할 수 있는 올바른 권한이 있는지 테스트 합니다.
간단한 테스트 객체를 S3 버킷에 업로드합니다. 이 명령이 실패하면 이 노트북에서 데이터 캡처 및 모델 모니터링 기능이 작동하지 않습니다. `s3:PutObject` 권한을 갖도록 이 노트북 인스턴스와 연결된 역할을 업데이트하고 이 유효성 검사를 다시 시도하여 이 문제를 해결할 수 있습니다.

In [11]:
# Upload some test files
S3Uploader.upload("./test_data/upload-test-file.txt", f"s3://{bucket}/test_upload")
print("Success! You are all set to proceed.")

Success! You are all set to proceed.


## Section 2 - Deploy pre-trained model with data capture enabled
 사전 훈련된 모델을 S3 버킷에 업로드하고, Amazon SageMaker 모델을 생성하고, Amazon SageMaker 실시간 엔드포인트를 생성하고, 엔드포인트에서 데이터 캡처를 활성화하여 엔드포인트 호출, 예측 및 메타데이터를 캡처합니다.

### 2.1 Upload the pre-trained model to S3
배포할 준비가 된 사전 훈련된 XGBoost 모델을 업로드합니다. 모델은 SageMaker에서 XGBoost 이미지를 사용하여 학습되었습니다. 
이 단계에서 사전 훈련된 자체 모델을 사용할 수도 있습니다. Amazon S3에 이미 사전 훈련된 모델이 있는 경우 s3_key를 지정하여 대신 추가할 수 있습니다.

In [12]:
##Upload the pretrained model to S3
s3_key = f"s3://{bucket}/{prefix}"
model_url = S3Uploader.upload("model/xgb-churn-prediction-model.tar.gz", s3_key)
model_url

's3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/xgb-churn-prediction-model.tar.gz'

### 2.2 Create SageMaker Model entity
S3에 업로드된 모델 파일에서 Amazon SageMaker 모델을 생성합니다.

In [13]:
model_name = f"pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H}"

image_uri = image_uris.retrieve(framework="xgboost", version="0.90-1", region=region)

model = Model(image_uri=image_uri, model_data=model_url, role=role)
print(model)

### 2.3 Deploy the model with data capture enabled
데이터 캡처가 활성화된 인스턴스에 SageMaker 모델을 배포합니다.

In [14]:
endpoint_name = f"model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H}"
print("EndpointName =", endpoint_name)

EndpointName = model-quality-monitor-2021-11-08-08


In [15]:
data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

-------------!

### 2.4 Create the SageMaker Predictor object from the endpoint to be used for invoking the model

In [16]:
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=session, serializer=CSVSerializer()
)

##  Section 3 - Generate a baseline for model quality performance
Validation Data를 사용하여 위에서 만든 Endpoint를 호출합니다. 이 데이터를 사용하여 배포된 모델의 예측은 Baseline으로 사용됩니다. 그 다음 SageMaker의 모델 모니터링을 사용하여 모델 성능 데이터를 계산하는 작업을 실행하고 Baseline을 기반으로 모델 품질 제약 조건을 제안합니다.

### 3.1 Execute predictions using the validation dataset
배포된 모델은 고객이 이탈할 확률을 반환합니다. cut-off 기준은 0.8로 설정하였습니다.

In [17]:
churn_cutoff = 0.8
validate_dataset = "validation_with_predictions.csv"

In [18]:
limit = 200  # Baseline을 계산하기 위해서 최소 200개의 샘플이 필요합니다.
i = 0
with open(f"test_data/{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # header
    with open("test_data/validation.csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            probability = float(predictor.predict(input_cols))
            prediction = "1" if probability > churn_cutoff else "0"
            baseline_file.write(f"{probability},{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            sleep(0.1)
print()
print("Done!")

........................................................................................................................................................................................................
Done!


### 3.2 Examine the predictions from the model

In [19]:
!head test_data/validation_with_predictions.csv

probability,prediction,label
0.01516005303710699,0,0
0.1684480607509613,0,0
0.21427156031131744,0,0
0.06330718100070953,0,0
0.02791607193648815,0,0
0.014169521629810333,0,0
0.00571369007229805,0,0
0.10534518957138062,0,0
0.025899196043610573,0,0


### 3.3 Upload the predictions as a baseline dataset
Validation Dataset을 사용하여 예측한 결과를 S3에 업로드하고 모델 품질 수준 및 기준 제약 조건을 생성하기 위한 기초 작업입니다.

In [20]:
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/baselining/data
Baseline results uri: s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/baselining/results


In [21]:
baseline_dataset_uri = S3Uploader.upload(f"test_data/{validate_dataset}", baseline_data_uri)
baseline_dataset_uri

's3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/baselining/data/validation_with_predictions.csv'

### 3.4 Create a baselining job with validation dataset predictions
모델 품질 모니터링 대상을 정의하고 모델 품질 모니터링 Baseline 생성을 실행합니다. 모델 모니터는 제공된 Validation Set을 기반으로 Baseline을 자동으로 생성합니다.

In [22]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

In [23]:
# Create the model quality monitoring object
churn_model_quality_monitor = ModelQualityMonitor(
    role=role,  # AWS IAM role
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1800,  ##Cron Job 주기보다 값이 작아야 함.
    sagemaker_session=session,
)

In [24]:
# Name of the model quality baseline job
baseline_job_name = f"model-quality-baseline-job-{datetime.utcnow():%Y-%m-%d-%H}"

In [25]:
# Execute the baseline suggestion job.
job = churn_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification", # 문제 유형을 지정하고 아래 필수 attribute의 header를 입력
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)
job.wait(logs=False)


Job Name:  model-quality-baseline-job-2021-11-08-09
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/baselining/data/validation_with_predictions.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/baselining/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..................................................................!

### 3.5 Explore the results of the baselining job
Baseline 정보가 S3에 업로드되는 것을 확인할 수 있습니다.

In [26]:
baseline_job = churn_model_quality_monitor.latest_baselining_job

In [27]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.571429,LessThanThreshold
precision,1,LessThanThreshold
accuracy,0.940299,LessThanThreshold
true_positive_rate,0.571429,LessThanThreshold
true_negative_rate,1,LessThanThreshold
false_positive_rate,0,GreaterThanThreshold
false_negative_rate,0.428571,GreaterThanThreshold
auc,0.939513,LessThanThreshold
f0_5,0.869565,LessThanThreshold
f1,0.727273,LessThanThreshold


* 위 결과에서는 모델의 F2 score가 0.625 미만에 대해 경고하도록 제약 조건을 설정한 것을 볼 수 있습니다. 
* Precision 처럼 강한 제약 조건을 가질 수도 있습니다. 모니터링에 사용하기 전에 필요에 따라 이 파일을 수정하는 것이 필요합니다.

## Section 4 - Setup continuous model monitoring to identify model quality drift
생성된 Baseline에 대해 배포된 모델의 품질을 모니터링하는 모델 모니터링 작업을 설정합니다.

### 4.1 Generate prediction data for Model Quality  Monitoring
아래 코드는 트래픽을 Endpoint로 보내기 위해 Thread를 시작합니다. 트래픽이 없으면 처리할 데이터가 없으므로 모니터링 작업은 `Failed`로 표시됩니다.
* Endpoint를 호출할 때 `inferenceId`를 반드시 설정하여야 합니다. 이 속성은 예측 데이터를 실측 데이터와 결합하는 데 사용됩니다.

In [28]:
def invoke_endpoint(ep_name, file_name):
    with open(file_name, "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )["Body"].read()
            i += 1
            sleep(1)


def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, "test_data/test-dataset-input-cols.csv")


thread = Thread(target=invoke_endpoint_forever)
thread.start()

### 4.2 View captured data
Amazon S3에 저장된 데이터 캡처 파일을 확인합니다. 호출이 발생한 시간을 기준으로 파일이 표시되며, S3 경로의 형식은 다음과 같습니다.

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [29]:
print("Waiting for captures to show up", end="")
for _ in range(120):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

Waiting for captures to show up............................................
Found Capture Files:
s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/datacapture/model-quality-monitor-2021-11-08-08/AllTraffic/2021/11/08/08/59-35-496-d99009c0-89a5-4505-ad8b-7fa1f73aca1f.jsonl
 s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/datacapture/model-quality-monitor-2021-11-08-08/AllTraffic/2021/11/08/09/11-38-907-b203250b-bc56-45b9-a4b1-6a886cc71f29.jsonl


In [30]:
# 'inferenceId' attribute that is used to join with ground truth data.
print(json.dumps(capture_record, indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "186,0.1,137.8,97,187.7,118,146.4,85,8.7,6,1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,1.1,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,0.51,0.52,0.53,1.2,1.3,0.54,1.4,0.55",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.01584203727543354",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "a0e7bb24-7771-46b2-807d-23ec8cc7dfc3",
    "inferenceId": "0",
    "inferenceTime": "2021-11-08T09:11:38Z"
  },
  "eventVersion": "0"
}


### 4.3 Start generating some fake ground truth
실제 사용 사례에서는 Ground truth 데이터를 정기적으로 수집하여 지정된 S3 위치에 업로드해야 합니다. 하지만 이 예제에서는 임의로 생성된 Ground truth를 활용하여 모델 품질 모니터링을 진행하도록 하겠습니다.

In [31]:
import random

def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    rand = random.random()
    return {
        "groundTruthData": {
            "data": "1" if rand < 0.3 else "0",  # randomly generate positive labels 30% of the time
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }

def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [32]:
NUM_GROUND_TRUTH_RECORDS = 200  # 200 are the number of rows in data we're sending for inference

def generate_fake_ground_truth_forever():
    j = 0
    while True:
        fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
        upload_ground_truth(fake_records, datetime.utcnow())
        j = (j + 1) % 5
        sleep(60 * 60)  # do this once an hour

gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 200 records to s3://sagemaker-ap-northeast-2-023376671800/sagemaker/ModelMonitor/ground_truth_data/2021-11-08-08/2021/11/08/09/1524.jsonl


### 4.4 Create a monitoring schedule
이제 기준 정보와 정답 레이블이 있으므로 모델 품질 모니터링 작업을 실행하기 위한 모니터링 스케줄을 만듭니다.

In [33]:
## Monitoring schedule name
churn_monitor_schedule_name = f"monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H}"

* 예측된 값을 해석하는 방법을 지정해야 합니다. 확률 값의 출력 위치는 첫번째 컬럼이므로 `"0"`을 지정, cut-off 값은 0.7로 임의 설정 하였습니다.
* `destination`으로는 `ModelQualityMonitor` 객체의 `LocalPath`를 지정해주어야 합니다. 

In [29]:
# Create an enpointInput
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.7,
    destination="/opt/ml/processing/input_data",
)

In [30]:
# Create the monitoring schedule to execute every hour.
from sagemaker.model_monitor import CronExpressionGenerator

response = churn_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=churn_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

In [31]:
# Check the monitoring schedule in the 'Scheduled' status
churn_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:ap-northeast-2:304701668247:monitoring-schedule/monitoring-schedule-2021-11-05-06',
 'MonitoringScheduleName': 'monitoring-schedule-2021-11-05-06',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2021, 11, 5, 6, 41, 15, 698000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 11, 5, 6, 41, 15, 721000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2021-11-05-06-41-15-459',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'model-quality-monitor-2021-11-05-06',
 'ResponseMetadata': {'RequestId': '2614bfbb-6678-4484-ada7-8ecf17541012',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2614bfbb-6678-4484-ada7-8ecf17541012',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '584',
   'date': 'Fri, 05 Nov 20

### 4.5 Examine monitoring schedule executions
첫 번째 실행이 정시에 발생하기 때문에 처음에는 실행이 없습니다. 실행은 1시간 후 최대 20분까지 실행되는 것이 일반적입니다.

In [32]:
executions = churn_model_quality_monitor.list_executions()
executions

No executions found for schedule. monitoring_schedule_name: monitoring-schedule-2021-11-05-06


[]

In [33]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = churn_model_quality_monitor.describe_schedule().get("LastMonitoringExecutionSummary")
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

Waiting for first execution
Execution found!


In [34]:
while not executions:
    executions = churn_model_quality_monitor.list_executions()
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

{'ProcessingInputs': [{'InputName': 'constraints',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-304701668247/sagemaker/ModelMonitor/baselining/results/constraints.json',
    'LocalPath': '/opt/ml/processing/baseline/constraints',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-304701668247/sagemaker/ModelMonitor/baselining/results/merge/model-quality-monitor-2021-11-05-06/AllTraffic/2021/11/05/06',
    'LocalPath': '/opt/ml/processing/input_data/model-quality-monitor-2021-11-05-06/AllTraffic/2021/11/05/06',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'result',
    'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-2-304701668247/s

### 4.6 Inspect a specific execution (latest execution)
* Completed - 모니터링 실행이 완료되었으며 위반 보고서에서 문제가 발견되지 않았음을 의미합니다.
* CompletedWithViolations - 실행이 완료되었지만 제약 조건 위반이 감지되었음을 의미합니다.
* Failed - 클라이언트 오류 또는 인프라 문제로 인해 모니터링 실행이 실패했습니다.
* Stopped - 작업이 최대 런타임을 초과했거나 수동으로 중지되었습니다.

In [35]:
status = execution["MonitoringExecutionStatus"]

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job["ProcessingJobStatus"])
    print(
        f"{latest_job['ProcessingJobName']} job exit message, if any:",
        latest_job.get("ExitMessage"),
    )
    print(
        f"{latest_job['ProcessingJobName']} job failure reason, if any:",
        latest_job.get("FailureReason"),
    )
    sleep(60)  # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = churn_model_quality_monitor.list_executions()[-1]
    execution = churn_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution["MonitoringExecutionStatus"]

print("Execution status is:", status)

if status != "Completed":
    print(execution)
    print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

Execution status is: CompletedWithViolations
{'MonitoringScheduleName': 'monitoring-schedule-2021-11-05-06', 'ScheduledTime': datetime.datetime(2021, 11, 5, 7, 0, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2021, 11, 5, 7, 3, 49, 687000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2021, 11, 5, 7, 15, 45, 874000, tzinfo=tzlocal()), 'MonitoringExecutionStatus': 'CompletedWithViolations', 'ProcessingJobArn': 'arn:aws:sagemaker:ap-northeast-2:304701668247:processing-job/model-quality-monitoring-202111050700-f4abc89c439469428bf181b3', 'EndpointName': 'model-quality-monitor-2021-11-05-06'}
====STOP==== 
 No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.


In [36]:
latest_execution = churn_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
print("Report Uri:", report_uri)

Report Uri: s3://sagemaker-ap-northeast-2-304701668247/sagemaker/ModelMonitor/baselining/results/model-quality-monitor-2021-11-05-06/monitoring-schedule-2021-11-05-06/2021/11/05/07


### 4.7 View violations generated by monitoring schedule
Baseline과 비교하여 위반이 있는 경우 S3에 업로드된 Report에 표시됩니다.

In [43]:
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

,constraint_check_type,description,metric_name
0,LessThanThreshold,Metric auc with 0.5283596122131285 +/- 0.0105902315698309 was LessThanThreshold '0.9395127993393898',auc
1,LessThanThreshold,Metric precision with 0.3333333333333333 +/- 0.01862456281627294 was LessThanThreshold '1.0',precision
2,LessThanThreshold,Metric truePositiveRate with 0.09465020576131687 +/- 0.0031985592089746944 was LessThanThreshold '0.5714285714285714',truePositiveRate
3,LessThanThreshold,Metric f1 with 0.14743589743589744 +/- 0.005034285255406367 was LessThanThreshold '0.7272727272727273',f1
4,LessThanThreshold,Metric accuracy with 0.6576576576576577 +/- 0.009261753638607663 was LessThanThreshold '0.9402985074626866',accuracy
5,GreaterThanThreshold,Metric falsePositiveRate with 0.08614232209737828 +/- 0.005937813811224641 was GreaterThanThreshold '0.0',falsePositiveRate
6,LessThanThreshold,Metric trueNegativeRate with 0.9138576779026217 +/- 0.005937813811224677 was LessThanThreshold '1.0',trueNegativeRate
7,GreaterThanThreshold,Metric falseNegativeRate with 0.9053497942386831 +/- 0.003198559208974675 was GreaterThanThreshold '0.4285714285714286',falseNegativeRate
8,LessThanThreshold,Metric recall with 0.09465020576131687 +/- 0.0031985592089746944 was LessThanThreshold '0.5714285714285714',recall
9,LessThanThreshold,Metric f2 with 0.11047070124879924 +/- 0.003704668132874108 was LessThanThreshold '0.625',f2


## Section 5 - Analyze model quality CloudWatch metrics
모니터링 스케줄은 CloudWatch 지표도 출력합니다. 이 섹션에서는 생성된 지표를 보고 모델 품질이 기준 임계값에서 벗어날 때 트리거될 CloudWatch 알람을 설정합니다. 해당 작업은 CloudWatch에서 수동으로도 설정하는 것이 가능합니다.

### 5.1 List the CW metrics generated

In [38]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")

namespace = "aws/sagemaker/Endpoints/model-metrics" #어떤 모델 모니터를 사용했는지에 따라 경로가 다름

cw_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": churn_monitor_schedule_name},
]

In [39]:
# List metrics through the pagination interface
paginator = cw_client.get_paginator("list_metrics")

for response in paginator.paginate(Dimensions=cw_dimensions, Namespace=namespace):
    model_quality_metrics = response["Metrics"]
    for metric in model_quality_metrics:
        print(metric["MetricName"])

f0_5_best_constant_classifier
f2_best_constant_classifier
f1_best_constant_classifier
auc
precision
accuracy_best_constant_classifier
true_positive_rate
f1
accuracy
false_positive_rate
f0_5
true_negative_rate
false_negative_rate
recall_best_constant_classifier
precision_best_constant_classifier
recall
f2


### 5.2 Create a CloudWatch Alarm
모델의 accuracy 값이 지정한 임계값 아래로 떨어지면 알람을 생성합니다.

In [40]:
alarm_name = "Acccuracy Drift Monitoring"
alarm_desc = "Trigger an CloudWatch alarm when the accuracy drifts"
model_quality_accuracy_drift_threshold = 0.64
metric_name = "accuracy"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=cw_dimensions,
    Period=3600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=model_quality_accuracy_drift_threshold,
    ComparisonOperator="LessThanOrEqualToThreshold",
    TreatMissingData="missing",
)

{'ResponseMetadata': {'RequestId': 'de256834-9a79-4406-8ec9-b3d081db9db5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'de256834-9a79-4406-8ec9-b3d081db9db5',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Fri, 05 Nov 2021 07:41:29 GMT'},
  'RetryAttempts': 0}}

## Clean up

In [41]:
!aws sagemaker list-monitoring-schedules --region 'ap-northeast-2'

{
    "MonitoringScheduleSummaries": [
        {
            "MonitoringScheduleName": "monitoring-schedule-2021-11-05-06",
            "MonitoringScheduleArn": "arn:aws:sagemaker:ap-northeast-2:304701668247:monitoring-schedule/monitoring-schedule-2021-11-05-06",
            "CreationTime": 1636094475.698,
            "LastModifiedTime": 1636096545.89,
            "MonitoringScheduleStatus": "Scheduled",
            "EndpointName": "model-quality-monitor-2021-11-05-06",
            "MonitoringJobDefinitionName": "model-quality-job-definition-2021-11-05-06-41-15-459",
            "MonitoringType": "ModelQuality"
        }
    ]
}


In [ ]:
!aws sagemaker delete-monitoring-schedule --monitoring-schedule-name "monitoring-schedule-2021-11-05-06"

In [42]:
!aws sagemaker list-endpoints --region 'ap-northeast-2'

{
    "Endpoints": [
        {
            "EndpointName": "model-quality-monitor-2021-11-05-06",
            "EndpointArn": "arn:aws:sagemaker:ap-northeast-2:304701668247:endpoint/model-quality-monitor-2021-11-05-06",
            "CreationTime": 1636093675.959,
            "LastModifiedTime": 1636094051.071,
            "EndpointStatus": "InService"
        }
    ]
}


In [ ]:
!aws sagemaker delete-endpoint --endpoint-name "model-quality-monitor-2021-11-05-06"